In [1]:
import pandas as pd
import numpy as np
import math
import csv
print('pandas V: ' + str(pd.__version__))
print('numpy V: ' + str(np.__version__))

pandas V: 2.2.2
numpy V: 2.1.0


### Cargamos la base de datos INEGI

In [2]:
file = 'C:\\Users\\rock-\\Documents\\base_inegi\\iter_00_cpv2020_csv\\iter_00_cpv2020\\conjunto_de_datos\\conjunto_de_datos_iter_00CSV20.csv'

df = pd.read_csv(file, low_memory=False)

df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


#### Creamos en el DataFrame las claves estatales

In [3]:
def haz_clave(row):
    clave_entidad = str(row['ENTIDAD'])
    longitud_clave = len(clave_entidad)
    if longitud_clave == 1:
        clave = '0' + clave_entidad
    else:
        clave = clave_entidad
    return clave

df['CLAVE_ENTIDAD'] = df[['ENTIDAD']].apply(haz_clave, axis=1)

##### Información del DataFrame

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 287 entries, ENTIDAD to CLAVE_ENTIDAD
dtypes: int64(6), object(281)
memory usage: 428.4+ MB


#### Construimos un DataFrame para obtener solo las tuplas que tienen los conteos totales estatales

In [5]:
df_totalxentidad = df[(df['MUN']==0) & (df['LOC']==0)]

print(f'(Filas, Columnas) --> {df_totalxentidad.shape}')

df_totalxentidad.head()

(Filas, Columnas) --> (33, 287)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC,CLAVE_ENTIDAD
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*,00
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,236003,174089,98724,70126,6021,15323,128996,1711,*,01
2061,2,Baja California,0,Total de la entidad Baja California,0,Total de la Entidad,NaN,NaN,NaN,3769020,...,800189,618175,384011,216865,41223,38772,293529,9582,*,02
7627,3,Baja California Sur,0,Total de la entidad Baja California Sur,0,Total de la Entidad,NaN,NaN,NaN,798447,...,148723,136538,67961,36197,14508,8675,77223,2608,*,03
10188,4,Campeche,0,Total de la entidad Campeche,0,Total de la Entidad,NaN,NaN,NaN,928363,...,114020,151613,38508,17976,23627,36397,130361,12028,*,04


#### Construimos un DataFrame con solo los registros de población total (POBTOT)

In [6]:
df_pobtotxentidad = df_totalxentidad[['CLAVE_ENTIDAD','ENTIDAD', 'NOM_ENT', 'NOM_MUN', 'NOM_LOC', 'POBTOT']][1:].copy(deep=True)

df_pobtotxentidad.sort_values(by=['POBTOT'], inplace=True)

df_pobtotxentidad.head()

,CLAVE_ENTIDAD,ENTIDAD,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT
17137,06,6,Colima,Total de la entidad Colima,Total de la Entidad,731391
7627,03,3,Baja California Sur,Total de la entidad Baja California Sur,Total de la Entidad,798447
10188,04,4,Campeche,Total de la entidad Campeche,Total de la Entidad,928363
106291,18,18,Nayarit,Total de la entidad Nayarit,Total de la Entidad,1235456
166578,29,29,Tlaxcala,Total de la entidad Tlaxcala,Total de la Entidad,1342977


#### Obtenemos los intervalos

Dividimos el total de entidades en 10 partes. Esto genera 10 bin's con 3 o 4 entidades en cada uno.

De cada bin se obtiene el mínimo y máximo de la población total para generar los intervalos.

In [7]:
entidades_poblacion_array = df_pobtotxentidad[['CLAVE_ENTIDAD', 'POBTOT']].to_numpy()

entidades_poblacion_lst = np.array_split(entidades_poblacion_array, 10)

lim_inf = []
lim_sup = []
bin_cells = {}
for bin, lst in enumerate(entidades_poblacion_lst):
    # print(lst)
    cells = []
    for idx, element in enumerate(lst):
        # print(idx)
        cells.append(element[0])
        if idx == 0:
            # print(element[1])
            lim_inf.append(element[1])
        elif idx == len(lst)-1:
            # print(element[1])
            lim_sup.append(element[1])
    bin_cells.update({bin+1:cells})

#### Construimos el DataFrame con los datos manipulados

Este DataFrame servirá para insertar los datos en PostgreSQL

Se crea también un csv para referencia.

In [8]:
data = {'id': ["nextval('covariable_id_seq')"]*10, 
        'name': ['Población total']*10,
        'code': ['POBTOT']*10,
        'mesh': ['state']*10,
        'bin': [i for i in range(1,11)],
        'interval': [str(lim_inf) + ':' + str(lim_sup) for lim_inf,lim_sup in zip(lim_inf,lim_sup)],
        'lim_inf': lim_inf, 
        'lim_sup':lim_sup,
        'cells_state': [None]*10,
        'cells_mun': [None]*10,
        'cells_ageb': [None]*10}

df_data = pd.DataFrame(data)

df_data['cells_state'] = df_data['bin'].map(bin_cells)

df_data['cells_state'] = df_data['cells_state'].apply(lambda x: "{" + ', '.join(x) + "}")

quoteColumnsList = ['name','code','mesh','interval','cells_state']

df_data.to_csv('datos_pobtot.csv', index=False)

df_data

,id,name,code,mesh,bin,interval,lim_inf,lim_sup,cells_state,cells_mun,cells_ageb
0,nextval('covariable_id_seq'),Población total,POBTOT,state,1,731391:1235456,731391,1235456,"{06, 03, 04, 18}",None,None
1,nextval('covariable_id_seq'),Población total,POBTOT,state,2,1342977:1832650,1342977,1832650,"{29, 01, 32, 10}",None,None
2,nextval('covariable_id_seq'),Población total,POBTOT,state,3,1857985:2320898,1857985,2320898,"{23, 17, 31}",None,None
3,nextval('covariable_id_seq'),Población total,POBTOT,state,4,2368467:2822255,2368467,2822255,"{22, 27, 24}",None,None
4,nextval('covariable_id_seq'),Población total,POBTOT,state,5,2944840:3082841,2944840,3082841,"{26, 25, 13}",None,None
5,nextval('covariable_id_seq'),Población total,POBTOT,state,6,3146771:3540685,3146771,3540685,"{05, 28, 12}",None,None
6,nextval('covariable_id_seq'),Población total,POBTOT,state,7,3741869:4132148,3741869,4132148,"{08, 02, 20}",None,None
7,nextval('covariable_id_seq'),Población total,POBTOT,state,8,4748846:5784442,4748846,5784442,"{16, 07, 19}",None,None
8,nextval('covariable_id_seq'),Población total,POBTOT,state,9,6166934:8062579,6166934,8062579,"{11, 21, 30}",None,None
9,nextval('covariable_id_seq'),Población total,POBTOT,state,10,8348151:16992418,8348151,16992418,"{14, 09, 15}",None,None


#### Script SQL

Generamos el script para insertar los datos en PostgreSQL

In [9]:
# Función para generar el script SQL
def generate_sql_insert_script(df):
    script = ""
    for index, row in df.iterrows():
        values = (
            row['id'],
            f"'{row['name']}'",
            f"'{row['code']}'",
            f"'{row['mesh']}'",
            row['bin'],
            f"'{row['interval']}'",
            row['lim_inf'],
            row['lim_sup'],
            f"'{row['cells_state']}'",
            f"'{row['cells_mun']}'" if row['cells_mun'] is not None else 'NULL',
            f"'{row['cells_ageb']}'" if row['cells_ageb'] is not None else 'NULL'
        )
        script += f"INSERT INTO covariable (id, name, code, mesh, bin, interval, lim_inf, lim_sup, cells_state, cells_mun, cells_ageb) VALUES ({', '.join(map(str, values))});\n"
    
    return script

# Generar el script SQL
sql_script = generate_sql_insert_script(df_data)

# Guardar el script en un archivo .sql
with open("insert_datos_pobtot.sql", "w", encoding="utf-8") as file:
    file.write(sql_script)

print("Script SQL generado y guardado en 'insert_covariable.sql'")

Script SQL generado y guardado en 'insert_covariable.sql'


### Cargar nuevos datos

Se genera el script SQL de nuevos datos a partir de un CSV.

In [13]:
name = 'datos_inegi_mesh_state'
new_data_csv_file = f'{name}.csv'
new_data_sql_file = f'insert_{name}.sql'

new_data = pd.read_csv(new_data_csv_file)

new_data[['cells_mun', 'cells_ageb']] = None

sql_script = generate_sql_insert_script(new_data)

with open(f"{new_data_sql_file}", "w", encoding="utf-8") as file:
    file.write(sql_script)

print(f"Script SQL generado y guardado en 'insert_{name}.sql'")

Script SQL generado y guardado en 'insert_datos_inegi_mesh_state.sql'


### Primer intento

A continuación está la primera aproximación de creación de intervalos.

Estos intervalos se hicieron tomando 10 intervalos igualmente espaciados entre los valores mínimo y máximo de la columna 'POBTOT'.

In [11]:
pobtot = 'POBTOT'

maxi = df_totalxentidad[[pobtot]][1:].astype('int32').max()[pobtot]

mini = df_totalxentidad[[pobtot]][1:].astype('int32').min()[pobtot]

interval_values = np.linspace(mini, maxi, num=11)

lim_inf = []
lim_sup = []
for i,j in enumerate(interval_values):
    if i < len(interval_values)-1:
        lim_inf.append(math.ceil(j))
    if i > 0:
        lim_sup.append(math.floor(j))

data = {'name': ['Población total']*10,
        'code': [pobtot]*10,
        'mesh': ['state']*10,
        'bin': [i for i in range(1,11)],
        'interval': [str(lim_inf) + ':' + str(lim_sup) for lim_inf,lim_sup in zip(lim_inf,lim_sup)],
        'lim_inf': lim_inf, 
        'lim_sup':lim_sup,
        'cells_state': ['']*10,
        'cells_mun': ['']*10,
        'cells_ageb': ['']*10}

df_data = pd.DataFrame(data)

df_data

,name,code,mesh,bin,interval,lim_inf,lim_sup,cells_state,cells_mun,cells_ageb
0,Población total,POBTOT,state,1,731391:2357493,731391,2357493,,,
1,Población total,POBTOT,state,2,2357494:3983596,2357494,3983596,,,
2,Población total,POBTOT,state,3,3983597:5609699,3983597,5609699,,,
3,Población total,POBTOT,state,4,5609700:7235801,5609700,7235801,,,
4,Población total,POBTOT,state,5,7235802:8861904,7235802,8861904,,,
5,Población total,POBTOT,state,6,8861905:10488007,8861905,10488007,,,
6,Población total,POBTOT,state,7,10488008:12114109,10488008,12114109,,,
7,Población total,POBTOT,state,8,12114110:13740212,12114110,13740212,,,
8,Población total,POBTOT,state,9,13740213:15366315,13740213,15366315,,,
9,Población total,POBTOT,state,10,15366316:16992418,15366316,16992418,,,


In [12]:
from collections import defaultdict

def insert_cell(row, valor, clave, bin_clave):
    lim_inf = row['lim_inf']
    lim_sup = row['lim_sup']
    if valor >= lim_inf and valor <= lim_sup:
        bin_clave[row['bin']].append(clave)

# Inicializa el diccionario bin_clave
bin_clave = defaultdict(list)

# Itera sobre cada par clave-valor
for clave, valor in zip(df_totalxentidad['CLAVE_ENTIDAD'][1:], df_totalxentidad[pobtot][1:]):
    df_data.apply(lambda x: insert_cell(x, valor, clave, bin_clave), axis=1)

# Ahora bin_clave contendrá todos los pares 'bin': 'clave' que cumplieron la condición

df_data['cells_state'] = df_data['bin'].map(bin_clave)

df_data

,name,code,mesh,bin,interval,lim_inf,lim_sup,cells_state,cells_mun,cells_ageb
0,Población total,POBTOT,state,1,731391:2357493,731391,2357493,"[01, 03, 04, 06, 10, 17, 18, 23, 29, 31, 32]",,
1,Población total,POBTOT,state,2,2357494:3983596,2357494,3983596,"[02, 05, 08, 12, 13, 22, 24, 25, 26, 27, 28]",,
2,Población total,POBTOT,state,3,3983597:5609699,3983597,5609699,"[07, 16, 20]",,
3,Población total,POBTOT,state,4,5609700:7235801,5609700,7235801,"[11, 19, 21]",,
4,Población total,POBTOT,state,5,7235802:8861904,7235802,8861904,"[14, 30]",,
5,Población total,POBTOT,state,6,8861905:10488007,8861905,10488007,[09],,
6,Población total,POBTOT,state,7,10488008:12114109,10488008,12114109,[],,
7,Población total,POBTOT,state,8,12114110:13740212,12114110,13740212,[],,
8,Población total,POBTOT,state,9,13740213:15366315,13740213,15366315,[],,
9,Población total,POBTOT,state,10,15366316:16992418,15366316,16992418,[15],,
